In [5]:
# layer and block
import torch
from torch import nn
from torch.nn import functional as f

# create a test data
X = torch.rand(2, 20)
print(X)

tensor([[0.0026, 0.9660, 0.7406, 0.8116, 0.2081, 0.3626, 0.6210, 0.5437, 0.8135,
         0.9073, 0.0816, 0.1030, 0.0515, 0.8680, 0.1650, 0.3615, 0.6745, 0.0812,
         0.3723, 0.5914],
        [0.7617, 0.3930, 0.3635, 0.8710, 0.4179, 0.0435, 0.3794, 0.1799, 0.8731,
         0.7385, 0.1241, 0.4651, 0.3932, 0.3704, 0.8045, 0.0097, 0.0369, 0.2830,
         0.4269, 0.7453]])


In [6]:
# basic usage of squential
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
print(net(X))

tensor([[ 3.0620e-01, -8.1479e-02, -1.7422e-01, -1.1966e-01, -2.6504e-01,
         -1.7143e-01,  1.0597e-01,  6.9276e-02, -3.7697e-02, -5.6737e-03],
        [ 1.6414e-01,  1.1049e-02, -1.7023e-01, -1.4671e-01, -2.1039e-01,
         -2.0458e-01, -1.3002e-04, -1.3449e-01, -2.8038e-03,  2.2289e-02]],
       grad_fn=<AddmmBackward0>)


In [7]:
# customized block with arbitrary layers
# --------------------------------------

# Block 1: full connected layer
class MLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(f.relu(self.hidden(X)))
    
net = MLP()
print(net(X))

tensor([[-0.1995,  0.1781,  0.1064, -0.0460, -0.2364, -0.1100,  0.0382,  0.0466,
         -0.0532, -0.2014],
        [-0.3185,  0.2561,  0.0763, -0.0489, -0.1881, -0.0921, -0.0415,  0.0336,
         -0.1436, -0.1656]], grad_fn=<AddmmBackward0>)


In [8]:
# Block 2: customized sequences
class MySequential(nn.Module):
    def __init__(self, *args) -> None:
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module
    
    def forward(self, X):
        for net in self._modules.values():
            X = net(X)
        return X
    
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
print(net(X))

tensor([[-0.0673, -0.0860, -0.2318,  0.0573, -0.1608,  0.0356,  0.0756, -0.0374,
         -0.1062, -0.1630],
        [-0.1487,  0.0629, -0.0747,  0.1166, -0.1223,  0.0828, -0.0277,  0.0346,
         -0.0611, -0.1534]], grad_fn=<AddmmBackward0>)


In [9]:
# Block 3: python operation in layers
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.linear = nn.Linear(20, 20)
        self.rand_weight = torch.rand((20, 20), requires_grad=False)  # work as a constant
    
    def forward(self, X):
        X = self.linear(X)
        X = f.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()
    
net = FixedHiddenMLP()
print(net(X))

tensor(-0.0907, grad_fn=<SumBackward0>)


In [11]:
# Block 3: nested blocks and layers
class NestMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), 
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    
    def forward(self, X):
        return self.linear(self.net(X))
    
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0563, grad_fn=<SumBackward0>)